In [1]:
import os
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine,inspect

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy
import csv,sqlite3


In [2]:
con = sqlite3.connect("./db/flights_data.sqlite")


In [3]:
df = pd.read_csv("./db/flights_metadata.csv")
df.head(1)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Unnamed: 21
0,2013,1,9E,Pinnacle Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",139.0,37.0,13.04,0.8,...,9.24,2.0,5.0,1860.0,789.0,75.0,451.0,0.0,545.0,NaN


In [4]:
del df['Unnamed: 21']

In [5]:
df.head(1)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2013,1,9E,Pinnacle Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",139.0,37.0,13.04,0.8,...,0.0,9.24,2.0,5.0,1860.0,789.0,75.0,451.0,0.0,545.0


In [6]:
df = df.rename(columns = {" month":"month"," weather_ct":"weather_ct"," arr_delay":"arr_delay"," carrier_delay":"carrier_delay"})

In [7]:
df = df[["year","month","carrier","carrier_name","airport","airport_name","arr_flights","arr_del15","carrier_ct","weather_ct",
"nas_ct","security_ct","late_aircraft_ct","arr_cancelled","arr_diverted","arr_delay","carrier_delay",
"weather_delay","nas_delay","security_delay","late_aircraft_delay"]]

In [8]:
df.head(1)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2013,1,9E,Pinnacle Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",139.0,37.0,13.04,0.8,...,0.0,9.24,2.0,5.0,1860.0,789.0,75.0,451.0,0.0,545.0


In [84]:
df1 = pd.read_csv("./db/airport_codes.csv")
df1.head()

,locationID,Latitude,Longitude
0,ADK,51.8781,176.6461
1,AKK,56.9386,154.1825
2,Z13,60.9047,161.4225
3,AKI,60.9028,161.2306
4,AUK,62.6800,164.6600


In [87]:
df1 = df1.rename(columns = {"locationID":"airport"})
df1.head()


,airport,Latitude,Longitude
0,ADK,51.8781,176.6461
1,AKK,56.9386,154.1825
2,Z13,60.9047,161.4225
3,AKI,60.9028,161.2306
4,AUK,62.6800,164.6600


In [88]:
df2 = df.merge(df1)
# df2 = df.merge(df1,left_on = 'airport',right_on = 'locationID')
df2.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Latitude,Longitude
0,2013,1,9E,Pinnacle Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",139.0,37.0,13.04,0.80,...,2.0,5.0,1860.0,789.0,75.0,451.0,0.0,545.0,32.8969,97.0381
1,2013,1,AA,American Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",13125.0,2372.0,470.16,98.23,...,338.0,35.0,129258.0,34386.0,9641.0,35490.0,191.0,49550.0,32.8969,97.0381
2,2013,1,AS,Alaska Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",62.0,16.0,3.39,0.00,...,0.0,0.0,528.0,206.0,0.0,309.0,0.0,13.0,32.8969,97.0381
3,2013,1,B6,JetBlue Airways,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",89.0,34.0,4.67,0.00,...,0.0,1.0,1617.0,396.0,0.0,1077.0,0.0,144.0,32.8969,97.0381
4,2013,1,DL,Delta Air Lines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",415.0,55.0,22.03,2.31,...,4.0,2.0,2912.0,1110.0,281.0,685.0,0.0,836.0,32.8969,97.0381


In [89]:
df2.to_sql("flights_data", con =con, if_exists='replace', index=False)


In [90]:
df2.dtypes

year                     int64
month                    int64
carrier                 object
carrier_name            object
airport                 object
airport_name            object
arr_flights            float64
arr_del15              float64
carrier_ct             float64
weather_ct             float64
nas_ct                 float64
security_ct            float64
late_aircraft_ct       float64
arr_cancelled          float64
arr_diverted           float64
arr_delay              float64
carrier_delay          float64
weather_delay          float64
nas_delay              float64
security_delay         float64
late_aircraft_delay    float64
Latitude               float64
Longitude              float64
dtype: object

In [91]:
engine = create_engine("sqlite:///db/flights_data.sqlite", encoding='utf8')
conn = engine.connect()

In [92]:
inspector = inspect(engine)
inspector.get_table_names()
inspector.get_columns("flights_data")


[{'name': 'year',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'month',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'carrier',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'carrier_name',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'airport',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'airport_name',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'arr_flights',
  'type': REAL(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'arr_del15',
  'type': REAL(),
  'nullable': True,
  'default': None,
  'autoincrement': 'aut

In [93]:
session = Session(engine)

In [94]:
flight_data = pd.read_sql("SELECT * FROM flights_data",engine)
flight_data.head(2)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Latitude,Longitude
0,2013,1,9E,Pinnacle Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",139.0,37.0,13.04,0.80,...,2.0,5.0,1860.0,789.0,75.0,451.0,0.0,545.0,32.8969,97.0381
1,2013,1,AA,American Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",13125.0,2372.0,470.16,98.23,...,338.0,35.0,129258.0,34386.0,9641.0,35490.0,191.0,49550.0,32.8969,97.0381


In [101]:
flight_data.dtypes

year                     int64
month                    int64
carrier                 object
carrier_name            object
airport                 object
airport_name            object
arr_flights            float64
arr_del15              float64
carrier_ct             float64
weather_ct             float64
nas_ct                 float64
security_ct            float64
late_aircraft_ct       float64
arr_cancelled          float64
arr_diverted           float64
arr_delay              float64
carrier_delay          float64
weather_delay          float64
nas_delay              float64
security_delay         float64
late_aircraft_delay    float64
Latitude               float64
Longitude              float64
dtype: object